# Manufacturer Popularity Over Time

This notebook produces a charts of manufacturer popularity, based on number of sightings, over time:

- Absolute, based on number of sightings
- Relative share, based on share of annual sightings

Manufacturers with a number of sightings below a threshold are excluded.

In [6]:
%run ../pathutils.ipynb
%run ../definitions.ipynb
%run ../database.ipynb
%run ../export.ipynb
%run utils.ipynb

In [7]:
# Annual sightings threshold below which manufacturers are excluded from the results
sightings_threshold = 10

In [8]:
import matplotlib.pyplot as plt

def export_absolute_counts_chart(export_folder_path, data):
    # Create a new figure
    fig, ax = plt.subplots(figsize=(14, 7))

    # Plot the bar chart
    data.plot(
        kind="bar",
        stacked=True,
        ax=ax
    )

    # Set axis properties
    ax.set_title("Aircraft Sightings by Manufacturer (Counts)")
    ax.set_xlabel("")
    ax.set_ylabel("Number of Sightings")

    # Place the legend below the chart
    ax.legend(
        title="Manufacturer",
        loc="upper center",
        bbox_to_anchor=(0.5, -0.12),
        ncol=3,
        frameon=False
    )

    plt.tight_layout()

    # Export the chart
    export_chart(export_folder_path, f"manufacturer-popularity-counts", "png")

    # Close the plot
    plt.close(fig)


In [9]:
import matplotlib.pyplot as plt

def export_relative_share_chart(export_folder_path, data):
    # Create a new figure
    fig, ax = plt.subplots(figsize=(14, 7))

    # Plot the bar chart
    data.plot(
        kind="bar",
        stacked=True,
        ax=ax
    )

    # Set axis properties
    ax.set_title("Aircraft Sightings by Manufacturer (Relative Share)")
    ax.set_xlabel("")
    ax.set_ylabel("Share of Sightings")
    ax.set_ylim(0, 100)
    ax.yaxis.set_major_formatter('{x:.0f}%')

    # Place the legend below the chart
    ax.legend(
        title="Manufacturer",
        loc="upper center",
        bbox_to_anchor=(0.5, -0.12),
        ncol=3,
        frameon=False
    )

    plt.tight_layout()

    # Export the chart
    export_chart(export_folder_path, f"manufacturer-popularity-share", "png")

    # Close the plot
    plt.close(fig)


In [10]:
# Construct the query
query = construct_query("aircraft", "sightings.sql", {})

try:
    # Run the query to load the data - this raises a ValueError if there's no data
    df = query_data("aircraft", query)

    # Create a year column
    df["Year"] = df["Date"].dt.year

    # Display unique manufacturers
    df["Manufacturer"].unique()

    # Aggregate Sightings by Manufacturer and Year
    manufacturer_trends = (
        df.groupby(["Year", "Manufacturer"])
        .size()
        .reset_index(name="Sightings")
    )

    # Remove manufacturers that fall below the sightings threshold
    row_indices = manufacturer_trends[manufacturer_trends["Sightings"] < sightings_threshold].index
    manufacturer_trends.drop(row_indices, inplace = True) 

    # Create a pivot table of absolute sighting counts per manufacturer per year
    pivot_counts = (
        manufacturer_trends
        .pivot(index="Year", columns="Manufacturer", values="Sightings")
        .fillna(0)
        .sort_index()
    )

    # Normalize by total sightings per year
    total_per_year = df.groupby("Year").size().rename("Total")
    normalized = manufacturer_trends.merge(total_per_year, on="Year")
    normalized["Share"] = normalized["Sightings"] / normalized["Total"]

    # Create a pivot table of relative shares of sightings per manufacturer per year
    pivot_shares = (
        normalized
        .pivot(index="Year", columns="Manufacturer", values="Share")
        .fillna(0)
        .sort_index()
        * 100
    )

    # Export the data to excel
    export_folder_path = get_export_folder_path()
    export_to_spreadsheet(export_folder_path, "manufacturer-popularity.xlsx", {
        "Absolute Counts": pivot_counts,
        "Relative Share": pivot_shares
    })

    # Export the chart
    export_absolute_counts_chart(export_folder_path, pivot_counts)
    export_relative_share_chart(export_folder_path, pivot_shares)

except ValueError:
    print(f"WARNING: No data found")